In [66]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
# 数据集路径（包含中文路径示例）
data_path = r'D:\PycharmProjects\pythonProject\work\课程实践-数据\数据汇总'

# 初始化列表来存储图像和标签
features = []
process_image = []
labels = []
sum1 = 0
sum2 = 0

# 直方图均衡化和均值滤波处理
# 预处理函数，包括图像大小调整、HOG特征提取和归一化
def preprocess_image(image):
    global process_image
    # 调整图像大小
    #cv2.bilateralFilter(image, 5, 75, 75)
    #image,image=cv2.threshold(image, 120, 255, cv2.THRESH_BINARY)
    #image = cv2.equalizeHist(image)
    #image = cv2.Canny(image, 80, 255)
    image = cv2.resize(image, (224, 224))
    #cv2.imshow('Image', image)
    #cv2.waitKey(0)  # 等待按键按下
    # 计算HOG特征
    features, hog_image = hog(image, orientations=9, pixels_per_cell=(8,8),
                              cells_per_block=(2, 2), block_norm='L2-Hys',
                              visualize=True, transform_sqrt=True)
    # 对HOG图像进行归一化处理
    #hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return features

# 递归遍历文件夹找到所有图像
def find_images_in_folder(folder_path):
    global sum1
    global sum2
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.JPG')\
                     or file_name.endswith('.PNG') or file_name.endswith('.JPEG') or file_name.endswith('.jpeg'):
                file_path = os.path.join(root, file_name)
                # 读取图像
                image = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
                if image is not None:
                    # 预处理图像并提取HOG特征
                    hog_features = preprocess_image(image)
                    features.append(hog_features)
                     # 根据文件所在文件夹名的最后一个字符判断标签
                    last_character = root[-1]
                    # 根据文件名最后一个字符判断标签
                    if last_character == 'y':
                        sum1 = sum1 + 1
                        labels.append(1)  # 手写
                    else:
                        sum2 = sum2 + 1
                        labels.append(0)  # 非手写

# 开始遍历数据集文件夹
find_images_in_folder(data_path)

# 将列表转换为NumPy数组
images = np.array(features)
labels = np.array(labels)
print(sum1 , sum2)
print(f'找到的图像数量：{len(images)}')
print(f'找到的标签数量：{len(labels)}')

110 93
找到的图像数量：203
找到的标签数量：203


In [6]:
# 随便显示一张图像
#print(len(images[7]))
#cv2.imshow('Image', images[20].reshape(162, 162))
#print(f'Label: {labels[1]}')
#cv2.waitKey(0)  # 等待按键按下
#cv2.destroyAllWindows()
#print(len(images[1]))
#print(len(process_image[0][1]))

In [72]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA

# images 和 labels 已经准备好
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)
#sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.fit_transform(X_test)
# 初始化 SVM 分类器
clf = SVC(kernel='rbf')

# 训练模型
clf.fit(X_train, y_train)

# 预测测试集
y_pred = clf.predict(X_test)
# 打印每个样本的真实标签和预测标签
for i in range(len(y_test)):
    true_label = y_test[i]
    pred_label = y_pred[i]
    print(f"样本 {i+1}: 真实标签={true_label}, 预测标签={pred_label}")

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率：{accuracy:.4f}')

# 计算精确率
precision = precision_score(y_test, y_pred)
print(f'精确率：{precision:.4f}')

# 计算召回率
recall = recall_score(y_test, y_pred)
print(f'召回率：{recall:.4f}')

# 计算F1分数
f1 = f1_score(y_test, y_pred)
print(f'F1分数：{f1:.4f}')

样本 1: 真实标签=1, 预测标签=1
样本 2: 真实标签=1, 预测标签=1
样本 3: 真实标签=1, 预测标签=0
样本 4: 真实标签=0, 预测标签=0
样本 5: 真实标签=1, 预测标签=1
样本 6: 真实标签=0, 预测标签=0
样本 7: 真实标签=1, 预测标签=1
样本 8: 真实标签=1, 预测标签=1
样本 9: 真实标签=1, 预测标签=1
样本 10: 真实标签=1, 预测标签=1
样本 11: 真实标签=0, 预测标签=0
样本 12: 真实标签=0, 预测标签=1
样本 13: 真实标签=0, 预测标签=0
样本 14: 真实标签=0, 预测标签=1
样本 15: 真实标签=1, 预测标签=0
样本 16: 真实标签=0, 预测标签=0
样本 17: 真实标签=1, 预测标签=1
样本 18: 真实标签=0, 预测标签=0
样本 19: 真实标签=1, 预测标签=0
样本 20: 真实标签=1, 预测标签=0
样本 21: 真实标签=1, 预测标签=1
样本 22: 真实标签=0, 预测标签=0
样本 23: 真实标签=0, 预测标签=0
样本 24: 真实标签=0, 预测标签=0
样本 25: 真实标签=1, 预测标签=1
样本 26: 真实标签=1, 预测标签=1
样本 27: 真实标签=0, 预测标签=0
样本 28: 真实标签=0, 预测标签=0
样本 29: 真实标签=0, 预测标签=0
样本 30: 真实标签=1, 预测标签=0
样本 31: 真实标签=1, 预测标签=1
样本 32: 真实标签=1, 预测标签=1
样本 33: 真实标签=1, 预测标签=1
样本 34: 真实标签=1, 预测标签=1
样本 35: 真实标签=0, 预测标签=0
样本 36: 真实标签=0, 预测标签=0
样本 37: 真实标签=1, 预测标签=1
样本 38: 真实标签=1, 预测标签=0
样本 39: 真实标签=1, 预测标签=1
样本 40: 真实标签=0, 预测标签=0
样本 41: 真实标签=0, 预测标签=0
样本 42: 真实标签=1, 预测标签=1
样本 43: 真实标签=0, 预测标签=0
样本 44: 真实标签=0, 预测标签=0
样本 45: 真实标签=0, 预测标签=0
样本 46: 真实标签=0, 预测标签

In [87]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假设 features 和 labels 已经准备好
# 初始化 KNN 分类器
knn_classifier = KNeighborsClassifier(n_neighbors=2)

# 训练模型
knn_classifier.fit(X_train, y_train)

# 预测测试集
y_pred = knn_classifier.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率：{accuracy:.4f}')

# 计算精确率
precision = precision_score(y_test, y_pred)
print(f'精确率：{precision:.4f}')

# 计算召回率
recall = recall_score(y_test, y_pred)
print(f'召回率：{recall:.4f}')

# 计算F1分数
f1 = f1_score(y_test, y_pred)
print(f'F1分数：{f1:.4f}')


准确率：0.8361
精确率：0.9600
召回率：0.7273
F1分数：0.8276


In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假设 features 和 labels 已经准备好
# 初始化逻辑回归分类器
logistic_regression = LogisticRegression(max_iter=1000)

# 训练模型
logistic_regression.fit(X_train, y_train)

# 预测测试集
y_pred = logistic_regression.predict(X_test)

# 计算准确率
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率：{accuracy:.4f}')

# 计算精确率
precision = precision_score(y_test, y_pred)
print(f'精确率：{precision:.4f}')

# 计算召回率
recall = recall_score(y_test, y_pred)
print(f'召回率：{recall:.4f}')

# 计算F1分数
f1 = f1_score(y_test, y_pred)
print(f'F1分数：{f1:.4f}')


准确率：0.8525
精确率：0.9000
召回率：0.8182
F1分数：0.8571


In [84]:
from sklearn.model_selection import GridSearchCV

# SVM分类
svm_param_grid = {'C':[1], 'gamma': [0.001]}
svm = SVC()
svm_grid = GridSearchCV(svm, svm_param_grid, cv=5)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_
y_pred_svm = best_svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred_svm)
    
results = {'SVM': acc_svm,'SVM_params': svm_grid.best_params_}

In [20]:
print(results)

{'SVM': 0.8524590163934426, 'SVM_params': {'C': 1, 'gamma': 0.001}}


In [25]:
from sklearn.neural_network import MLPClassifier
# 将图像数据展平以适应MLP输入
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)
#X_train_flatten = PCA(n_components=50).fit_transform(X_train_flatten)
#X_test_flatten = PCA(n_components=50).fit_transform(X_test_flatten)
###X_train_flatten= scaler.transform(X_train_flatten)   # 转换数据集
#X_test_flatten = scaler.transform(X_test_flatten)
# 构建和训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(512,256,256,192,128), activation='relu', solver='adam',)
mlp.fit(X_train_flatten, y_train)
# 构建并训练分类器（MLP）
# 预测
y_pred = mlp.predict(X_test_flatten)
# 打印每个样本的真实标签和预测标签
for i in range(len(y_test)):
    true_label = y_test[i]
    pred_label = y_pred[i]
    print(f"样本 {i+1}: 真实标签={true_label}, 预测标签={pred_label}")
# 计算并打印评估指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'准确率：{accuracy:.4f}')
print(f'精确率：{precision:.4f}')
print(f'召回率：{recall:.4f}')
print(f'F1分数：{f1:.4f}')

样本 1: 真实标签=1, 预测标签=1
样本 2: 真实标签=1, 预测标签=1
样本 3: 真实标签=1, 预测标签=0
样本 4: 真实标签=0, 预测标签=0
样本 5: 真实标签=1, 预测标签=1
样本 6: 真实标签=0, 预测标签=0
样本 7: 真实标签=1, 预测标签=1
样本 8: 真实标签=1, 预测标签=1
样本 9: 真实标签=1, 预测标签=1
样本 10: 真实标签=1, 预测标签=1
样本 11: 真实标签=0, 预测标签=0
样本 12: 真实标签=0, 预测标签=1
样本 13: 真实标签=0, 预测标签=0
样本 14: 真实标签=0, 预测标签=1
样本 15: 真实标签=1, 预测标签=0
样本 16: 真实标签=0, 预测标签=0
样本 17: 真实标签=1, 预测标签=1
样本 18: 真实标签=0, 预测标签=0
样本 19: 真实标签=1, 预测标签=1
样本 20: 真实标签=1, 预测标签=1
样本 21: 真实标签=1, 预测标签=1
样本 22: 真实标签=0, 预测标签=0
样本 23: 真实标签=0, 预测标签=0
样本 24: 真实标签=0, 预测标签=0
样本 25: 真实标签=1, 预测标签=1
样本 26: 真实标签=1, 预测标签=1
样本 27: 真实标签=0, 预测标签=0
样本 28: 真实标签=0, 预测标签=0
样本 29: 真实标签=0, 预测标签=0
样本 30: 真实标签=1, 预测标签=1
样本 31: 真实标签=1, 预测标签=1
样本 32: 真实标签=1, 预测标签=1
样本 33: 真实标签=1, 预测标签=1
样本 34: 真实标签=1, 预测标签=1
样本 35: 真实标签=0, 预测标签=0
样本 36: 真实标签=0, 预测标签=0
样本 37: 真实标签=1, 预测标签=1
样本 38: 真实标签=1, 预测标签=0
样本 39: 真实标签=1, 预测标签=1
样本 40: 真实标签=0, 预测标签=0
样本 41: 真实标签=0, 预测标签=0
样本 42: 真实标签=1, 预测标签=1
样本 43: 真实标签=0, 预测标签=0
样本 44: 真实标签=0, 预测标签=0
样本 45: 真实标签=0, 预测标签=0
样本 46: 真实标签=0, 预测标签

In [42]:
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(n_estimators=8)
clf.fit(X_train, y_train)
# 预测测试集
y_pred = clf.predict(X_test)
# 打印每个样本的真实标签和预测标签
for i in range(len(y_test)):
    true_label = y_test[i]
    pred_label = y_pred[i]
    print(f"样本 {i+1}: 真实标签={true_label}, 预测标签={pred_label}")

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率：{accuracy:.4f}')

# 计算精确率
precision = precision_score(y_test, y_pred)
print(f'精确率：{precision:.4f}')

# 计算召回率
recall = recall_score(y_test, y_pred)
print(f'召回率：{recall:.4f}')

# 计算F1分数
f1 = f1_score(y_test, y_pred)
print(f'F1分数：{f1:.4f}')

样本 1: 真实标签=1, 预测标签=1
样本 2: 真实标签=1, 预测标签=1
样本 3: 真实标签=1, 预测标签=0
样本 4: 真实标签=0, 预测标签=0
样本 5: 真实标签=1, 预测标签=1
样本 6: 真实标签=0, 预测标签=0
样本 7: 真实标签=1, 预测标签=1
样本 8: 真实标签=1, 预测标签=1
样本 9: 真实标签=1, 预测标签=1
样本 10: 真实标签=1, 预测标签=0
样本 11: 真实标签=0, 预测标签=0
样本 12: 真实标签=0, 预测标签=1
样本 13: 真实标签=0, 预测标签=0
样本 14: 真实标签=0, 预测标签=1
样本 15: 真实标签=1, 预测标签=1
样本 16: 真实标签=0, 预测标签=0
样本 17: 真实标签=1, 预测标签=1
样本 18: 真实标签=0, 预测标签=0
样本 19: 真实标签=1, 预测标签=0
样本 20: 真实标签=1, 预测标签=0
样本 21: 真实标签=1, 预测标签=1
样本 22: 真实标签=0, 预测标签=0
样本 23: 真实标签=0, 预测标签=0
样本 24: 真实标签=0, 预测标签=1
样本 25: 真实标签=1, 预测标签=1
样本 26: 真实标签=1, 预测标签=1
样本 27: 真实标签=0, 预测标签=0
样本 28: 真实标签=0, 预测标签=0
样本 29: 真实标签=0, 预测标签=0
样本 30: 真实标签=1, 预测标签=1
样本 31: 真实标签=1, 预测标签=0
样本 32: 真实标签=1, 预测标签=1
样本 33: 真实标签=1, 预测标签=1
样本 34: 真实标签=1, 预测标签=1
样本 35: 真实标签=0, 预测标签=0
样本 36: 真实标签=0, 预测标签=0
样本 37: 真实标签=1, 预测标签=1
样本 38: 真实标签=1, 预测标签=0
样本 39: 真实标签=1, 预测标签=1
样本 40: 真实标签=0, 预测标签=0
样本 41: 真实标签=0, 预测标签=0
样本 42: 真实标签=1, 预测标签=1
样本 43: 真实标签=0, 预测标签=0
样本 44: 真实标签=0, 预测标签=0
样本 45: 真实标签=0, 预测标签=0
样本 46: 真实标签=0, 预测标签

ModuleNotFoundError: No module named 'tensorflow'

In [47]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

# 数据集路径
data_path = r'D:\PycharmProjects\pythonProject\work\课程实践-数据\数据汇总'

# 初始化列表来存储图像特征和标签
labels = []
# 初始化列表来存储文本特征
text_features = []
features = []
# 递归遍历文件夹找到所有图像
def find_images_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.JPG')\
                     or file_name.endswith('.PNG') or file_name.endswith('.JPEG') or file_name.endswith('.jpeg'):
                file_path = os.path.join(root, file_name)
                # 读取图像
                image = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
                if image is not None:
                    # 预处理图像并提取HOG特征
                    hog_features = preprocess_image(image)
                    features.append(hog_features)
                    # 提取类别标签
                    folder_name = os.path.basename(root)
                    category = int(folder_name.split('_')[0])  # 文件夹前的数字对应类别，类别从1开始
                    #print(file_path)
                    labels.append(category)
        for file_name in files:
            if file_name.endswith('.txt'):
                file_path = os.path.join(root, file_name)
                #print(file_path)
                with open(file_path, 'r',encoding='utf-8') as file:
                    text_ = file.read()
                    text_features.append(text_)

# 开始遍历数据集文件夹
find_images_in_folder(data_path)
labels = np.array(labels)
images = np.array(features)
text = np.array(text_features)
print(f'找到的图像数量：{len(images)}')

找到的图像数量：203


In [48]:
for label in labels:
    print(label)
print(f'找到的图像数量：{len(features)}')
print(len(labels))

10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
1
1
1
1
1
1
1
1
1
1
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
5
5
5
5
5
6
6
6
6
7
7
7
7
7
8
9
找到的图像数量：203
203


In [49]:
import re
import os
import gensim
from gensim import corpora, models
import numpy as np
import jieba

# 使用jieba进行分词，并去除无效符号
def clean_text(_text):
    _text = re.sub(r'[^\u4e00-\u9fa5]', '', _text)  # 保留中文字符
    # 过滤单个字的分词
    return _text

# 将文档分词
texts = [[word for word in jieba.lcut(clean_text(document))] for document in text]
print(texts)
# 统计各文档单词，生成词典
dictionary = corpora.Dictionary(texts)
# 去掉出现次数过少的单词
min_freq = 5  # 设置最低频率
dictionary.filter_extremes(no_below=min_freq,no_above=79)
print(dictionary)
# 利用词典将文档转化成词频表示的向量
corpus = [dictionary.doc2bow(text) for text in texts]

# 将词频表示的向量转化成TF-IDF表示的向量
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 将TF-IDF表示的向量转化成LSI表示的向量
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=13)
corpus_lsi = lsi[corpus_tfidf]

# 将LSI表示的向量转化为numpy数组
text_features = gensim.matutils.corpus2dense(corpus_lsi, num_terms=13).T

print(f'找到的文本数量：{len(text_features)}')
print(f'每个LSI特征的长度：{text_features.shape[1]}')

[['帕累托', '分布', '是', '以', '意大利', '经济学家', '维', '弗雷', '多', '帕雷托', '命名', '的', '是从', '大量', '真实世界', '的', '现象', '中', '发现', '的', '幂', '定律', '分布', '帕累托', '发现', '绝大部分', '社会', '财富', '最终', '总会', '流向', '少数', '人群', '他', '还', '发现', '这种', '不', '平衡', '模式', '可以', '重复', '出现', '甚至', '可以', '预测'], ['泊松', '分布', '是', '一种', '统计', '与', '概率', '学里', '常见', '到', '的', '离散', '机率', '分布', '由', '二项分布', '推导', '而', '来', '适合', '用于', '描述', '单位', '时间', '或', '单位', '面积', '内', '随机', '事件', '的', '平均', '发生', '次数'], ['泊松', '分布', '是', '一种', '离散', '概率分布', '用于', '描述', '在', '固定', '时间', '或', '空间', '范围', '内', '某个', '事件', '发生', '的', '次数', '其', '概率分布', '函数', '为', '其中', '表示', '事件', '发生', '的', '次数', '表示', '单位', '时间', '或', '空间', '内该', '事件', '的', '平均', '发生率'], ['当', '一个', '随机', '事件', '以', '固定', '的', '平均', '瞬时', '速率', '或', '密度', '随机', '且', '独立', '地', '出现', '时', '那么', '这个', '事件', '在', '单位', '时间', '面积', '或', '体积', '内', '出现', '的', '次数', '或', '个数', '就', '近似', '地', '服从', '泊松', '分布'], ['泊松', '概率分布', '是', '考虑', '在', '连续', '时间', '和', '空间', '单位', '上', '

In [65]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 合并图像特征和文本特征
features = np.hstack((images, text_features))

# images 和 labels 已经准备好
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)


# 使用最佳参数初始化模型
best_clf = SVC(kernel='rbf')

# 训练模型
best_clf.fit(X_train, y_train)

# 预测测试集
y_pred = best_clf.predict(X_test)

# 打印每个样本的真实标签和预测标签
for i in range(len(y_test)):
    true_label = y_test[i]
    pred_label = y_pred[i]
    print(f"样本 {i+1}: 真实标签={true_label}, 预测标签={pred_label}")

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率：{accuracy:.4f}')

样本 1: 真实标签=11, 预测标签=12
样本 2: 真实标签=11, 预测标签=12
样本 3: 真实标签=12, 预测标签=12
样本 4: 真实标签=12, 预测标签=12
样本 5: 真实标签=11, 预测标签=12
样本 6: 真实标签=11, 预测标签=11
样本 7: 真实标签=12, 预测标签=12
样本 8: 真实标签=3, 预测标签=12
样本 9: 真实标签=7, 预测标签=12
样本 10: 真实标签=3, 预测标签=12
样本 11: 真实标签=12, 预测标签=12
样本 12: 真实标签=2, 预测标签=12
样本 13: 真实标签=3, 预测标签=11
样本 14: 真实标签=5, 预测标签=12
样本 15: 真实标签=11, 预测标签=11
样本 16: 真实标签=5, 预测标签=11
样本 17: 真实标签=11, 预测标签=12
样本 18: 真实标签=12, 预测标签=12
样本 19: 真实标签=11, 预测标签=11
样本 20: 真实标签=12, 预测标签=12
样本 21: 真实标签=3, 预测标签=12
样本 22: 真实标签=12, 预测标签=12
样本 23: 真实标签=12, 预测标签=12
样本 24: 真实标签=11, 预测标签=11
样本 25: 真实标签=12, 预测标签=12
样本 26: 真实标签=11, 预测标签=12
样本 27: 真实标签=3, 预测标签=11
样本 28: 真实标签=12, 预测标签=12
样本 29: 真实标签=2, 预测标签=11
样本 30: 真实标签=12, 预测标签=12
样本 31: 真实标签=11, 预测标签=12
样本 32: 真实标签=11, 预测标签=12
样本 33: 真实标签=12, 预测标签=12
样本 34: 真实标签=11, 预测标签=12
样本 35: 真实标签=12, 预测标签=11
样本 36: 真实标签=12, 预测标签=12
样本 37: 真实标签=12, 预测标签=12
样本 38: 真实标签=12, 预测标签=11
样本 39: 真实标签=2, 预测标签=12
样本 40: 真实标签=12, 预测标签=12
样本 41: 真实标签=12, 预测标签=12
样本 42: 真实标签=7, 预测标签=12
样本 43: 真实标签=1

In [63]:
# 标准化特征
from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(len(X_train[1]))

# 构建并训练分类器（MLP）
mlp_classifier = MLPClassifier(hidden_layer_sizes=(2048,1024,),solver='adam', random_state=42)
mlp_classifier.fit(X_train, y_train)

y_pred = mlp_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'分类器的准确度：{accuracy * 100:.2f}%')

13
分类器的准确度：86.89%
